In [1]:
import numpy as np
from scipy import stats
from scipy.optimize import minimize

In [15]:
x = [1, 1, 0, 1, 1, 0, 0]
y = [1, 0, 0, 0, 1, 0, 1]

In [17]:
for xx, yy in zip(x, y):
    print xx, yy

1 1
1 0
0 0
1 0
1 1
0 0
0 1


In [18]:
def fun(theta):
    p = 1
    
    for i in range(len(x)):
        p *= theta[0] if x[i] + y[i] == 0 else theta[3] if x[i] + y[i] == 2 else theta[1] if x[i] == 0 else theta[2]
        
    return p

cons = (
    {'type': 'eq', 'fun': lambda theta:  1 - np.sum(theta)},
    {'type': 'ineq', 'fun': lambda theta:  theta[0]},
    {'type': 'ineq', 'fun': lambda theta:  theta[1]},
    {'type': 'ineq', 'fun': lambda theta:  theta[2]},
    {'type': 'ineq', 'fun': lambda theta:  theta[3]}
)

In [19]:
minimize(fun, [1., 1., 1., 1.], method = 'SLSQP', constraints = cons)

  status: 0
 success: True
    njev: 2
    nfev: 12
     fun: 4.7940365872048111e-94
       x: array([  4.44089210e-16,   1.00000000e+00,   2.22044605e-16,
         2.22044605e-16])
 message: 'Optimization terminated successfully.'
     jac: array([  3.62227185e-71,   4.79403659e-94,   1.44890870e-70,
         1.44890870e-70,   0.00000000e+00])
     nit: 2